In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.table import Table, join, vstack
from astropy import wcs
from astropy.io import fits, ascii
from astropy.coordinates import SkyCoord

## Generate Tables 1-4 for Wong et al. (2022) ApJ paper

In [2]:
datadir = 'images/'
analdir = 'struct/'

In [3]:
clust = True

In [4]:
for line in ['12', '13']:
    # Get lists of trunks, branches, leaves
    for i, typ in enumerate(['trunks', 'branches', 'leaves', 'clusters']):
        label = ['T', 'B', 'L', 'C']
        txtfile = os.path.join(analdir,'30Dor_feather_mosaic_1p8_'+line+'_'+typ+'.txt')
        col1 = np.loadtxt(txtfile, usecols=0, dtype=int)
        addtab = Table([col1,[label[i]]*len(col1)], names=('_idx','dendrotyp'))
        if i == 0 or clust:
            typtab = addtab
        elif i < 3:
            typtab = vstack([typtab, addtab])
    typtab.sort('_idx')
    # Get dendro table
    dtab = Table.read(analdir+'30Dor_feather_mosaic_1p8_'+line+'_full_catalog.txt', format='ascii.ecsv')
    # Get properties table
    ptab = Table.read(analdir+'30Dor_feather_mosaic_1p8_'+line+'_physprop_resolve.txt', format='ascii.ecsv')
    # Merge the tables and select only clusters
    ctab = join(dtab, ptab)
    ctab = join(ctab, typtab)
    print('Length of merged table:',len(ctab))
    # Mark unresolved structures
    ctab = ctab[ctab['vrms_k']>0]
    ctab['dendrotyp'].description = 'Type of structure [(T)runk,(B)ranch,(L)eaf]'
    # Convert to log10 units
    if line == '12':
        docols = ['rad_pc_dcon', 'vrms_k', 'mlumco', 'mvir', 'alpha']
    else:
        docols = ['rad_pc_dcon', 'vrms_k', 'mlte', 'mvir', 'alphalte']
    for col in docols:
        print('Min/max for',col,'is',np.min(ctab[col]),np.max(ctab[col]))
        if col in ctab.colnames:
            ctab[col] = np.log10(ctab[col])
            ctab['e_'+col] = ctab['e_'+col] / np.log(10)
            ctab['e_'+col].unit = ctab[col].unit
    # Add the RA, DEC, and velocity columns
    header = fits.getheader(datadir+'30Dor_feather_mosaic_'+line+'CO_12meter.image.fits.gz')
    wcsobj = wcs.WCS(header)
    pixcoo = list(zip(ctab['x_cen'], ctab['y_cen'], ctab['v_cen']))
    ra_array  = wcsobj.all_pix2world(pixcoo,0).T[0]
    dc_array  = wcsobj.all_pix2world(pixcoo,0).T[1]
    vel_array = wcsobj.all_pix2world(pixcoo,0).T[2]/1000.
    coord = SkyCoord(ra_array, dc_array, unit='deg')
    ra_str = coord.ra.to_string(u.hour, sep=':', precision=2, pad=True)
    dc_str = coord.dec.to_string(u.degree, sep=':', precision=1, pad=True, alwayssign=True)
    ctab.add_columns([ra_array, dc_array, vel_array, ra_str, dc_str], 
                     names=['RAdeg', 'DECdeg', 'vLSR', 'RAhms', 'DECdms'],
                     indexes=[1, 1, 1, 1, 1])
    ctab['RAdeg'].unit = 'degree'
    ctab['RAdeg'].description = 'Center R.A. (J2000)'
    ctab['DECdeg'].unit = 'degree'
    ctab['DECdeg'].description = 'Center Decl. (J2000)'
    ctab['vLSR'].unit = 'km/s'
    ctab['vLSR'].description = 'Mean LSR velocity'
    # Add the area in pc2 and alpha_lte
    ctab['area_pc2'] = (ctab['area_exact']*(5e4*u.pc)**2).to(u.pc**2,equivalencies=u.dimensionless_angles())
    ctab['area_pc2'].description = 'Projected area of structure'
    ctab['major_sigma'].description = 'rms major axis width'
    ctab['minor_sigma'].description = 'rms minor axis width'
    ctab['rad_pc_dcon'].description = 'log10 of deconvolved radius in pc'
    ctab['e_rad_pc_dcon'].description = 'error in rad_pc_dcon'
    ctab['vrms_k'].description = 'log10 of rms linewidth in km/s'
    ctab['e_vrms_k'].description = 'error in vrms_k'
    ctab['mlumco'].description = 'log10 of CO-based mass scaling XGal by 1.6'
    ctab['e_mlumco'].description = 'error in mlumco'
    ctab['mlte'].description = 'log10 of LTE mass using H2/13CO=3e6'
    ctab['e_mlte'].description = 'error in mlte'
    ctab['mvir'].description = 'log10 of virial mass using deconvolved radius'
    ctab['e_mvir'].description = 'error in mvir'
    ctab['alpha'].description = 'log10 of virial parameter from mvir and mlumco'
    ctab['e_alpha'].description = 'error in alpha'
    ctab['alphalte'].description = 'log10 of virial parameter from mvir and mlte'
    ctab['e_alphalte'].description = 'error in alphalte'
    ctab['position_angle'].description = 'position angle measured CCW from west'
    # Sort and add a counter
    ctab.sort('RAdeg')
    ctab['cnt'] = 1+np.array(range(len(ctab)))
    ctab['cnt'].description = 'Structure number in RA order'
    print('Columns in merged table:',ctab.colnames)
    if line == '12':
        mrt_tab = Table([ctab['cnt'], ctab['RAdeg'], ctab['DECdeg'], ctab['vLSR'], ctab['flux12'],
                         ctab['major_sigma'], ctab['minor_sigma'], ctab['position_angle'], 
                         ctab['area_pc2'], ctab['rad_pc_dcon'], ctab['e_rad_pc_dcon'], 
                         ctab['vrms_k'], ctab['e_vrms_k'], ctab['mlumco'], ctab['e_mlumco'], 
                         ctab['mvir'], ctab['e_mvir'], ctab['alpha'], ctab['e_alpha'],
                         ctab['refdist'], ctab['dendrotyp']])
        pub_tab = Table([ctab['cnt'], ctab['RAhms'], ctab['DECdms'], ctab['vLSR'], ctab['flux12'],
                         ctab['major_sigma'], ctab['minor_sigma'], ctab['position_angle'], 
                         ctab['area_pc2'], ctab['rad_pc_dcon'], ctab['e_rad_pc_dcon'], 
                         ctab['vrms_k'], ctab['e_vrms_k'], ctab['mlumco'], ctab['e_mlumco'], 
                         ctab['mvir'], ctab['e_mvir'], ctab['alpha'], ctab['e_alpha'],
                         ctab['refdist'], ctab['dendrotyp']])
    else:
        mrt_tab = Table([ctab['cnt'], ctab['RAdeg'], ctab['DECdeg'], ctab['vLSR'], ctab['flux13'],
                         ctab['major_sigma'], ctab['minor_sigma'], ctab['position_angle'], 
                         ctab['area_pc2'], ctab['rad_pc_dcon'], ctab['e_rad_pc_dcon'], 
                         ctab['vrms_k'], ctab['e_vrms_k'], ctab['mlte'], ctab['e_mlte'], 
                         ctab['mvir'], ctab['e_mvir'], ctab['alphalte'], ctab['e_alphalte'],
                         ctab['refdist'], ctab['dendrotyp']])
        pub_tab = Table([ctab['cnt'], ctab['RAhms'], ctab['DECdms'], ctab['vLSR'], ctab['flux13'],
                         ctab['major_sigma'], ctab['minor_sigma'], ctab['position_angle'], 
                         ctab['area_pc2'], ctab['rad_pc_dcon'], ctab['e_rad_pc_dcon'], 
                         ctab['vrms_k'], ctab['e_vrms_k'], ctab['mlte'], ctab['e_mlte'],
                         ctab['mvir'], ctab['e_mvir'], ctab['alphalte'], ctab['e_alphalte'],
                         ctab['refdist'], ctab['dendrotyp']])
    if clust:
        outfile = 'clusters'
        mrt_tab.remove_column('dendrotyp')
        pub_tab.remove_column('dendrotyp')
    else:
        outfile = 'dendros'
    # LaTeX table for paper
    for column in pub_tab.colnames:
        if column.endswith('deg'):
            pub_tab[column].info.format = '.5f'
        elif column.endswith('ms'):
            pub_tab[column].info.format = '%s'
        elif column == 'dendrotyp':
            pub_tab[column].info.format = '%s'
        elif column == 'cnt':
            pub_tab[column].info.format = '%3d'
        elif column == 'position_angle' or column == 'refdist':
            pub_tab[column].info.format = '.0f'
        else:
            pub_tab[column].info.format = '.2f'
    pub_tab[:10].write(sys.stdout, format='latex')
    # Machine readable table
    for column in mrt_tab.colnames:
        mrt_tab[column].fill_value = ascii.masked
        if column.endswith('deg'):
            mrt_tab[column].info.format = '.5f'
        elif column == 'cnt':
            mrt_tab[column].info.format = '%3d'
        elif column == 'dendrotyp':
            pub_tab[column].info.format = '%s'
        elif 'rad_pc' in column or 'vrms_k' in column or 'alpha' in column:
            mrt_tab[column].info.format = '.3f'
        else:
            mrt_tab[column].info.format = '.2f'
    mrt_tab.write(outfile+line+'.txt', format='mrt', overwrite=True)
    # Compare area measures
#     x = np.log10(ctab['area_pc2'])
#     y = np.log10(np.pi*ctab['rad_pc']**2)
#     plt.scatter(x, y)
#     polyco = np.polyfit(x, y, 1)
#     xmod = np.linspace(-0.5, 2.1, num=10)
#     plt.plot(xmod, np.polyval(polyco,xmod), 'r--')
#     print('Fit coefficients:',polyco)
#     print('equiv area larger than exact by',10**polyco[1])
#     plt.gca().set_aspect('equal')
#     plt.xlabel('area_exact')
#     plt.ylabel('$\pi R^2$')
#     plt.show()

Length of merged table: 142
Min/max for rad_pc_dcon is 0.14485433711009224 9.541004754242312
Min/max for vrms_k is 0.40208183036685147 3.9134925074275877
Min/max for mlumco is 4.21010618824482 28004.835579184495
Min/max for mvir is 50.16788359788971 82468.33805567656
Min/max for alpha is 1.1778644736764199 134.81323295987445


Set OBSGEO-B to   -23.022886 from OBSGEO-[XYZ].
Set OBSGEO-H to     5053.796 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


Columns in merged table: ['_idx', 'RAdeg', 'DECdeg', 'vLSR', 'RAhms', 'DECdms', 'area_ellipse', 'area_exact', 'flux', 'major_sigma', 'minor_sigma', 'position_angle', 'radius', 'v_cen', 'v_rms', 'x_cen', 'y_cen', 'tmax', 'tpkav', 'area_pc2', 'rad_pc', 'e_rad_pc', 'rad_pc_dcon', 'e_rad_pc_dcon', 'vrms_k', 'e_vrms_k', 'axratio', 'e_axratio', 'mlumco', 'e_mlumco', 'siglum', 'e_siglum', 'mvir_raw', 'e_mvir_raw', 'mvir', 'e_mvir', 'sigvir', 'e_sigvir', 'alpha', 'e_alpha', 'refdist', 'flux12', 'flux13', 'mlte', 'siglte', 'e_mlte', 'e_siglte', 'alphalte', 'e_alphalte', 'dendrotyp', 'cnt']
\begin{table}
\begin{tabular}{cccccccccccccccccccc}
cnt & RAhms & DECdms & vLSR & flux12 & major_sigma & minor_sigma & position_angle & area_pc2 & rad_pc_dcon & e_rad_pc_dcon & vrms_k & e_vrms_k & mlumco & e_mlumco & mvir & e_mvir & alpha & e_alpha & refdist \\
 &  &  & $\mathrm{km\,s^{-1}}$ & $\mathrm{Jy\,km\,s^{-1}}$ & $\mathrm{{}^{\prime\prime}}$ & $\mathrm{{}^{\prime\prime}}$ & $\mathrm{{}^{\circ}}$ & $\m

Set OBSGEO-B to   -23.022886 from OBSGEO-[XYZ].
Set OBSGEO-H to     5053.796 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [5]:
pub_tab

cnt,RAhms,DECdms,vLSR,flux13,major_sigma,minor_sigma,position_angle,area_pc2,rad_pc_dcon,e_rad_pc_dcon,vrms_k,e_vrms_k,mlte,e_mlte,mvir,e_mvir,alphalte,e_alphalte,refdist
,,,km / s,Jy km / s,arcsec,arcsec,deg,pc2,pc,pc,km / s,km / s,solMass,solMass,solMass,solMass,,,arcsec
int64,str11,str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,05:38:22.44,-69:03:51.5,253.36,10.88,3.78,1.04,-153,5.04,-0.12,0.05,0.14,0.04,2.85,0.04,3.23,0.08,0.37,0.09,169
2,05:38:22.60,-69:06:43.6,254.62,6.98,3.47,1.09,-161,4.07,-0.12,0.04,-0.44,0.04,2.69,0.04,2.06,0.08,-0.62,0.09,113
3,05:38:23.16,-69:03:26.8,250.56,4.04,1.91,1.36,-171,2.57,-0.18,0.04,0.26,0.04,2.41,0.04,3.39,0.08,0.98,0.09,187
4,05:38:25.81,-69:03:02.5,252.03,55.22,4.81,2.61,-157,13.94,0.20,0.04,0.18,0.04,3.59,0.04,3.63,0.08,0.04,0.09,201
5,05:38:26.30,-69:01:45.6,247.89,3.65,3.16,1.57,-154,3.13,-0.02,0.04,-0.10,0.04,2.37,0.04,2.84,0.08,0.47,0.09,272
6,05:38:26.90,-69:01:36.3,246.08,2.51,1.94,1.11,55,2.12,-0.25,0.06,-0.32,0.05,2.23,0.04,2.18,0.10,-0.04,0.10,279
7,05:38:27.11,-69:02:38.5,250.61,31.76,7.85,3.31,-164,14.23,0.37,0.04,0.08,0.04,3.33,0.04,3.59,0.08,0.26,0.09,220
8,05:38:27.18,-69:02:53.8,253.33,12.55,3.89,1.74,-137,4.79,0.06,0.04,0.02,0.04,2.93,0.04,3.16,0.08,0.23,0.09,206
